In [1]:
!pip install llama-index-llms-groq
!pip install llama-index llama-index-llms-huggingface llama-index-embeddings-huggingface transformers accelerate bitsandbytes llama-index-readers-web matplotlib flash-attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) 

### Load data

In [2]:
from llama_index.readers.web import BeautifulSoupWebReader

url = "https://www.theverge.com/2023/9/29/23895675/ai-bot-social-network-openai-meta-chatbots"
docs = BeautifulSoupWebReader().load_data([url])

### LLM + Embeddings

In [3]:
## LLM

from llama_index.llms.groq import Groq
from google.colab import userdata

llm = Groq(
    model = "llama3-70b-8192",
    api_key = userdata.get('GROQ_API_KEY')
)

In [5]:
## Embeddings

from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

Settings.llm = llm
Settings.embed_model = HuggingFaceEmbedding(
    model_name = "BAAI/bge-small-en-v1.5"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Index

In [6]:
from llama_index.core import VectorStoreIndex
from llama_index.core import SummaryIndex

vector_index = VectorStoreIndex.from_documents(docs)
summary_index = SummaryIndex.from_documents(docs)

In [7]:
from llama_index.core.response.notebook_utils import display_response

import logging
import sys

logging.basicConfig(
    stream=sys.stdout,
    level=logging.INFO
)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

### Basic Query Engine

In [8]:
query_engine = vector_index.as_query_engine(
    response_mode="compact"
)

response = query_engine.query("How do OpenAI and Meta differ on AI tools?")
display_response(response)

**`Final Response:`** OpenAI tends to present its products as productivity tools, whereas Meta is in the entertainment business.

In [9]:
query_engine = vector_index.as_query_engine(
    response_mode="compact"
)

response = query_engine.query("What are the new features added by OpenAI to ChatGPT?")
display_response(response)

**`Final Response:`** According to the context, the new features added by OpenAI to ChatGPT include the ability to interact with its large language model via voice and the ability to upload images and ask questions about them.

### Router Query Engine

In [10]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

vector_tool = QueryEngineTool(
    vector_index.as_query_engine(),
    metadata = ToolMetadata(
        name = 'vector_search',
        description = "Useful for searching for specific facts.",
    ),
)

summary_tool = QueryEngineTool(
    summary_index.as_query_engine(response_mode = "tree_summarize"),
    metadata = ToolMetadata(
        name = "summary",
        description = "Useful for summarizing an entire document.",
    ),
)

In [11]:
Settings.debug = True

#### Single Selector

In [13]:
from llama_index.core.query_engine import RouterQueryEngine

query_engine = RouterQueryEngine.from_defaults(
    [vector_tool, summary_tool],
    select_multi=False,
    verbose=True,
)

response = query_engine.query("What was mentioned about Meta?")
display_response(response)

Selecting query engine 0: The question asks about a specific fact mentioned about Meta, so it's useful for searching for specific facts..


**`Final Response:`** Meta is building LLMs (Large Language Models) and has found its own uses for generative AI and voices. The company unveiled 28 personality-driven chatbots to be used in Meta's messaging apps, with celebrities like Charli D'Amelio, Dwyane Wade, Kendall Jenner, MrBeast, Snoop Dogg, Tom Brady, and Paris Hilton lending their voices to the effort.

In [14]:
response = query_engine.query(
    "What was mentioned about Meta? What other companies are mentioned and what was said about them?"
)
display_response(response)

Selecting query engine 0: The question is asking for specific facts about Meta and other companies, so a summary method that is useful for searching for specific facts is most relevant..


**`Final Response:`** According to the context, Meta was mentioned as a company that is building Large Language Models (LLMs) and has found its own uses for generative AI and voices. Specifically, Meta unveiled 28 personality-driven chatbots to be used in its messaging apps, with celebrities like Charli D'Amelio, Dwyane Wade, and Snoop Dogg lending their voices to the effort.

Other companies mentioned in the context are OpenAI and Adobe. OpenAI was mentioned as a company that tends to present its products as productivity tools, and its ChatGPT was discussed as a tool that has become more powerful with the addition of voice features. Adobe was mentioned in the context of Figma's CEO discussing life after the company's failed sale to Adobe.

### Multi Selector

In [15]:
from llama_index.core.query_engine import RouterQueryEngine

query_engine = RouterQueryEngine.from_defaults(
    [vector_tool, summary_tool],
    select_multi = True,
    verbose = True
)
response = query_engine.query(
    "What was mentioned about Meta? What other companies are mentioned and what was said about them?"
)
display_response(response)

Selecting query engine 0: This choice is relevant because the question is asking for specific facts about Meta and other companies, which requires searching for specific information..


**`Final Response:`** According to the context, Meta was mentioned as a company that is building Large Language Models (LLMs) and has found its own uses for generative AI and voices. Specifically, Meta unveiled 28 personality-driven chatbots to be used in its messaging apps, with celebrities like Charli D'Amelio, Dwyane Wade, and Snoop Dogg lending their voices to the effort.

Other companies mentioned in the context are OpenAI and Adobe. OpenAI was mentioned as a company that tends to present its products as productivity tools, and its ChatGPT was discussed as a tool that has become more powerful with the addition of voice interactions. Adobe was mentioned in the context of Figma's CEO discussing life after the company's failed sale to Adobe.

In [16]:
response = query_engine.query(
    "Summarize what was mentioned about OpenAI?"
)
display_response(response)

Selecting query engine 1: The question asks to summarize what was mentioned about OpenAI, which implies summarizing an entire document or text..


**`Final Response:`** According to the article, OpenAI announced updates for ChatGPT, including a feature that allows users to interact with its large language model via voice, and another feature that lets users upload images and ask questions about them. This makes ChatGPT feel more powerful and gives it a hint of personality. The voice feature is currently available to ChatGPT Plus subscribers, and free users won't be able to use it for some time. The article suggests that this technology could lead to a synthetic companion that is smarter, more patient, more empathetic, and more available than human companions.